In [55]:
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
import string


from sklearn import svm
from sklearn.model_selection import train_test_split



In [56]:
train_df = pd.read_csv("train.csv")

In [57]:
train_df.head(5)

,id,tid1,tid2,title1_en,title2_en,label
0,195611,0,1,There are two new old-age insurance benefits f...,"Police disprove ""bird's nest congress each per...",unrelated
1,191474,2,3,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outstrips Hong Kong? Shenzhen S...,unrelated
2,25300,2,4,"""If you do not come to Shenzhen, sooner or lat...",The GDP overtopped Hong Kong? Shenzhen clarifi...,unrelated
3,123757,2,8,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP overtakes Hong Kong? Bureau of ...,unrelated
4,141761,2,11,"""If you do not come to Shenzhen, sooner or lat...",Shenzhen's GDP outpaces Hong Kong? Defending R...,unrelated


In [58]:
def clean_title(title):
  return ''.join([word for word in title.lower() if word not in string.punctuation])

In [59]:
def preprocess(df):
  hm = {}
  metrics = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan']
  for index, row in df.iterrows():
    title1 = clean_title(row['title1_en'])
    title2 = clean_title(row['title2_en'])
    vector = CountVectorizer().fit_transform([title1, title2]).toarray()

    for similarity in metrics:
      if similarity not in hm:
        hm[similarity] = []
      hm[similarity].append(pairwise_distances(vector, metric=similarity)[0][1])

  for sim in metrics:
    df[sim] = hm[sim]
  
  del df['title1_en']
  del df['title2_en']

  return df


In [ ]:
df = preprocess(train_df)
print(df)
X, y = df.loc[:, df.columns != 'label'], df['label']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)




clf = svm.SVC()
clf.fit(X_train, y_train)

            id    tid1    tid2      label  cityblock    cosine  euclidean  \
0       195611       0       1  unrelated       31.0  0.885668   5.567764   
1       191474       2       3  unrelated       35.0  0.778596   6.244998   
2        25300       2       4  unrelated       30.0  0.728837   5.830952   
3       123757       2       8  unrelated       38.0  0.871376   6.633250   
4       141761       2      11  unrelated       35.0  0.857305   6.403124   
...        ...     ...     ...        ...        ...       ...        ...   
256437  113364  167562   48447  unrelated       21.0  1.000000   5.196152   
256438   49407  167562   49795  unrelated       24.0  1.000000   5.477226   
256439  130134  167562  114783  unrelated       27.0  0.891852   5.744563   
256440  101494  167562  137705  unrelated       26.0  0.881530   5.477226   
256441   89356  167563   66480  unrelated       21.0  0.924622   5.000000   

          l1        l2  manhattan  
0       31.0  5.567764       31.0  
1  

In [ ]:
pred = clf.predict(X_val)
pred

In [ ]:
clf.score(X_val, y_val)